<a href="https://colab.research.google.com/github/RautRitesh/langgraph/blob/main/multi_agentic_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### This textbook consist of multi agentic system